# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 9:50AM,243243,10,BPS1986,Beach Patient Assistance,Released
1,Aug 2 2022 9:50AM,243243,10,BPS1987,Beach Patient Assistance,Released
2,Aug 2 2022 9:50AM,243243,10,BPS1988,Beach Patient Assistance,Released
3,Aug 2 2022 9:50AM,243243,10,BPS1989,Beach Patient Assistance,Released
4,Aug 2 2022 9:50AM,243243,10,BPS1990,Beach Patient Assistance,Released
5,Aug 2 2022 9:50AM,243243,10,BPS2000,Beach Patient Assistance,Released
6,Aug 2 2022 9:34AM,243241,19,TR-RC-08032022,"GCH Granules USA, Inc.",Released
7,Aug 2 2022 9:23AM,243240,12,HH6491,Hush Hush,Released
8,Aug 2 2022 9:23AM,243240,12,HH-36201,Hush Hush,Released
9,Aug 2 2022 9:23AM,243240,12,HH6492,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,243237,Released,64
30,243239,Released,58
31,243240,Released,3
32,243241,Released,1
33,243243,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243237,NaN,NaN,64.0
243239,NaN,NaN,58.0
243240,NaN,NaN,3.0
243241,NaN,NaN,1.0
243243,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243152,0.0,1.0,1.0
243170,8.0,37.0,0.0
243185,0.0,0.0,1.0
243200,0.0,0.0,1.0
243201,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243152,0,1,1
243170,8,37,0
243185,0,0,1
243200,0,0,1
243201,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243152,0,1,1
1,243170,8,37,0
2,243185,0,0,1
3,243200,0,0,1
4,243201,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243152,,1,1
1,243170,8,37,
2,243185,,,1
3,243200,,,1
4,243201,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 9:50AM,243243,10,Beach Patient Assistance
6,Aug 2 2022 9:34AM,243241,19,"GCH Granules USA, Inc."
7,Aug 2 2022 9:23AM,243240,12,Hush Hush
10,Aug 2 2022 9:21AM,243239,10,ISDIN Corporation
68,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation
132,Aug 2 2022 9:02AM,243227,20,"ACI Healthcare USA, Inc."
134,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation
142,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated"
145,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc."
146,Aug 2 2022 8:30AM,243202,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 2 2022 9:50AM,243243,10,Beach Patient Assistance,,,6
1,Aug 2 2022 9:34AM,243241,19,"GCH Granules USA, Inc.",,,1
2,Aug 2 2022 9:23AM,243240,12,Hush Hush,,,3
3,Aug 2 2022 9:21AM,243239,10,ISDIN Corporation,,,58
4,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,,,64
5,Aug 2 2022 9:02AM,243227,20,"ACI Healthcare USA, Inc.",,1,1
6,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,,,8
7,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated",,,3
8,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc.",,,1
9,Aug 2 2022 8:30AM,243202,10,Emerginnova,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 9:50AM,243243,10,Beach Patient Assistance,6,,
1,Aug 2 2022 9:34AM,243241,19,"GCH Granules USA, Inc.",1,,
2,Aug 2 2022 9:23AM,243240,12,Hush Hush,3,,
3,Aug 2 2022 9:21AM,243239,10,ISDIN Corporation,58,,
4,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,64,,
5,Aug 2 2022 9:02AM,243227,20,"ACI Healthcare USA, Inc.",1,1,
6,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,8,,
7,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated",3,,
8,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc.",1,,
9,Aug 2 2022 8:30AM,243202,10,Emerginnova,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 9:50AM,243243,10,Beach Patient Assistance,6,,
1,Aug 2 2022 9:34AM,243241,19,"GCH Granules USA, Inc.",1,,
2,Aug 2 2022 9:23AM,243240,12,Hush Hush,3,,
3,Aug 2 2022 9:21AM,243239,10,ISDIN Corporation,58,,
4,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,64,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 9:50AM,243243,10,Beach Patient Assistance,6.0,NaN,NaN
1,Aug 2 2022 9:34AM,243241,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
2,Aug 2 2022 9:23AM,243240,12,Hush Hush,3.0,NaN,NaN
3,Aug 2 2022 9:21AM,243239,10,ISDIN Corporation,58.0,NaN,NaN
4,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,64.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 9:50AM,243243,10,Beach Patient Assistance,6.0,0.0,0.0
1,Aug 2 2022 9:34AM,243241,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
2,Aug 2 2022 9:23AM,243240,12,Hush Hush,3.0,0.0,0.0
3,Aug 2 2022 9:21AM,243239,10,ISDIN Corporation,58.0,0.0,0.0
4,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,64.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3891506,164.0,5.0,0.0
12,243240,3.0,0.0,0.0
16,243233,3.0,1.0,0.0
18,243232,1.0,0.0,0.0
19,1216031,23.0,3.0,0.0
20,486397,1.0,38.0,8.0
21,243218,1.0,0.0,0.0
22,243214,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3891506,164.0,5.0,0.0
1,12,243240,3.0,0.0,0.0
2,16,243233,3.0,1.0,0.0
3,18,243232,1.0,0.0,0.0
4,19,1216031,23.0,3.0,0.0
5,20,486397,1.0,38.0,8.0
6,21,243218,1.0,0.0,0.0
7,22,243214,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,164.0,5.0,0.0
1,12,3.0,0.0,0.0
2,16,3.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,23.0,3.0,0.0
5,20,1.0,38.0,8.0
6,21,1.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,164.0
1,12,Released,3.0
2,16,Released,3.0
3,18,Released,1.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
Executing,5.0,0.0,1.0,0.0,3.0,38.0,0.0,1.0
Released,164.0,3.0,3.0,1.0,23.0,1.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,5.0,0.0,1.0,0.0,3.0,38.0,0.0,1.0
2,Released,164.0,3.0,3.0,1.0,23.0,1.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,5.0,0.0,1.0,0.0,3.0,38.0,0.0,1.0
2,Released,164.0,3.0,3.0,1.0,23.0,1.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()